In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from Utils.U_net_Modules import *

In [3]:
model = torchvision.models.resnet50(pretrained = True)

c:\Users\FeldmarschallSanjith\Desktop\BaseLineUnet\BaseLine-UNet-IMGDec\newenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\FeldmarschallSanjith\Desktop\BaseLineUnet\BaseLine-UNet-IMGDec\newenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
new_model = nn.Sequential(*list(model.children())[:-4])

In [6]:
print(new_model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [7]:
inp = torch.randn((3,224,224))

In [8]:
out = new_model(inp.unsqueeze(0))

In [9]:
out.shape

torch.Size([1, 512, 28, 28])

In [10]:
out = out.flatten()

In [11]:
out.shape

torch.Size([401408])

In [12]:
class BayarConv(nn.Module):

    def __init__(self, in_channel, out_channel, kernel_size = 5, stride = 1, padding = 0):
        super(BayarConv, self).__init__()

        self.in_channel = in_channel
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.mid_ele = torch.ones(self.in_channel, self.out_channel, 1) * -1.000

        self.kernel = nn.Parameter(torch.randn((self.in_channel,self.out_channel,kernel_size**2-1)), requires_grad= True)


    def constraint(self):
        self.kernel.data = self.kernel.data.div(self.kernel.data.sum(dim = -1, keepdim=True))
        center = self.kernel_size**2//2
        real_kernel = torch.cat((self.kernel[:,:,:center], self.mid_ele, self.kernel[:,:,center:]),dim=2)
        real_kernel = real_kernel.reshape((self.out_channel, self.in_channel, self.kernel_size, self.kernel_size))
        return real_kernel
    
    def forward(self, x):
        return F.conv2d(x, self.constraint(), stride = self.stride, padding=self.padding)


class BayarConvBlock(nn.Module):

    def __init__(self, in_channel = 12, out_channel = 1024, stride = 1, padding = 0):

        super(BayarConvBlock, self).__init__()

        self.in_channel = in_channel
        self.out_channel = out_channel
        self.padding = padding
        self.stride = stride

        self.mod = nn.Sequential(
            nn.Conv2d(self.in_channel,64,7,stride = 2),
            nn.MaxPool2d(kernel_size= 2),
            nn.ReLU(inplace= True),
            nn.Conv2d(64,128,3,stride = self.stride),
            nn.MaxPool2d(kernel_size= 2),
            nn.ReLU(inplace= True),
            nn.Conv2d(128,256, 5,stride = self.stride),
            nn.MaxPool2d(kernel_size= 2),
            nn.ReLU(inplace= True),
            nn.Conv2d(256,self.out_channel, 3,stride = self.stride),
            nn.MaxPool2d(kernel_size= 2),
            nn.ReLU(inplace= True)
        )

    def forward(self, x):

        return self.mod(x)


class BayarBlock(nn.Module):

    def __init__(self, in_channel = 3, out_channel = 1024, kernel_size = 3, stride = 1, padding = 0):

        super(BayarBlock, self).__init__()

        self.in_channel = in_channel
        self.out_channel = out_channel
        self.padding = padding
        self.kernel_size = kernel_size
        self.stride = stride

        self.Bayarconv = BayarConv(self.in_channel, 12)
        self.rest = BayarConvBlock(12, self.out_channel)

    def forward(self, x):

        x = self.Bayarconv(x)
        return self.rest(x)

In [13]:
mod = BayarBlock(3, 1024)

In [14]:
out1 = mod(inp.unsqueeze(0))

In [15]:
out1.shape

torch.Size([1, 1024, 4, 4])

In [22]:
class ResNetInterMediateResult(nn.Module):

    def __init__(self, model = torchvision.models.resnet50(pretrained = True)):
        super(ResNetInterMediateResult,self).__init__()
        
        self.model = model
        self.layers = nn.Sequential(*list(self.model.children())[:-1]) 
        self.inter_results = []

    def forward(self, x):
        for layer_name, layer in self.layers._modules.items():
            x = layer(x)
            self.inter_results.append((layer_name, x))

        return self.inter_results

c:\Users\FeldmarschallSanjith\Desktop\BaseLineUnet\BaseLine-UNet-IMGDec\newenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\FeldmarschallSanjith\Desktop\BaseLineUnet\BaseLine-UNet-IMGDec\newenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
my_mod = ResNetInterMediateResult()

In [24]:
my_mod.eval()

ResNetInterMediateResult(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequent

In [25]:
res = my_mod(inp.unsqueeze(0))

In [26]:
len(res)

9

In [29]:
res[2]

('2',
 tensor([[[[2.2706e-01, 0.0000e+00, 0.0000e+00,  ..., 1.2049e-01,
            2.0314e-01, 3.6480e-01],
           [0.0000e+00, 3.3524e-01, 3.5168e-01,  ..., 0.0000e+00,
            0.0000e+00, 2.3075e-01],
           [5.1993e-01, 0.0000e+00, 0.0000e+00,  ..., 3.8414e-01,
            3.9785e-01, 2.9145e-01],
           ...,
           [2.5395e-01, 0.0000e+00, 0.0000e+00,  ..., 4.3608e-01,
            6.9133e-02, 6.5801e-02],
           [5.0557e-01, 6.8376e-01, 4.4945e-01,  ..., 3.4191e-01,
            5.8033e-01, 2.3186e-01],
           [1.0628e-01, 2.5106e-01, 4.5638e-01,  ..., 5.1242e-02,
            3.1455e-01, 4.9321e-01]],
 
          [[8.7771e-01, 9.7381e-01, 6.4616e-01,  ..., 8.1012e-01,
            7.5664e-01, 7.3520e-01],
           [8.0770e-01, 1.0615e+00, 9.2167e-01,  ..., 5.4262e-01,
            6.8386e-01, 8.5785e-01],
           [8.1811e-01, 1.0753e+00, 8.5629e-01,  ..., 6.0140e-01,
            1.0494e+00, 1.0619e+00],
           ...,
           [1.6904e-01, 2.6327e-